In [2]:
import datetime as dt
import time
import warnings

import matplotlib.pyplot as plt
import geopandas as gpd
import pandas as pd
from shapely.geometry import Point

from GoogleEarth import GoogleEarth
from Simulator import Simulator

warnings.filterwarnings("ignore")

In [ ]:
source = -80.52, 40.84
start_time = dt.datetime(2023, 2, 3, 21)
end_time = dt.datetime(2023, 2, 6, 0)
iteration_interval = dt.timedelta(hours=1)
simulator_interval = dt.timedelta(seconds=1200)
ge = GoogleEarth()
simulator = Simulator(start_time, end_time, source, iteration_interval, simulator_interval, ge, 1)

In [ ]:
simulator = Simulator(start_time, end_time, source, iteration_interval, simulator_interval, ge, 1)
for _ in range(24 * 20):
    t = time.time()
    simulator.step()
    
    status_time = simulator.time
    status = simulator.get_status()
    status_time = status_time.strftime("%Y%m%d %H%M%S")
    plt.savefig(f"../simulation/{status_time}.png")
    status.to_csv(f"../simulation/{status_time}.csv")
    plt.close()
    print(f"It takes {round(time.time() - t, 3)} to simulate {status_time} status of size {len(simulator.status)}")
    

In [6]:
df_shp = gpd.read_file("../data/geoshape/cb_2018_us_cbsa_20m/cb_2018_us_cbsa_20m.shp")
df_geo = pd.read_csv("../simulation/simulation.csv", sep="\t")[["Longitude", "Latitude", "EST", "P"]]
geometry = [Point(xy) for xy in zip(df_geo["Longitude"], df_geo["Latitude"])]
df_geo = gpd.GeoDataFrame(df_geo, geometry=geometry, crs="EPSG:4269")
df_geo = gpd.sjoin(df_geo, df_shp, predicate="within", how="left")
df_geo.index = pd.to_datetime(df_geo["EST"]).dt.strftime("%Y%m%d")
df_geo.index.name = "Date"

In [8]:
df_geo.to_pickle("../simulation/SimulationGeo.pkl", compression="gzip")